In [1]:
import copy
import sys
from IJCAI2017_TOOL import *

In [161]:
#读入题目给出的商家信息表
SHOP_INFO_EN = pd.read_csv('./Flamingo_Rank4_code/data_new/SHOP_INFO_EN.csv')
SHOP_INFO_EN.head()

,SHOP_ID,CITY,SHOP_LOC,SHOP_PAY,SHOP_SCO,SHOP_COM,SHOP_LEV,SHOP_CA1,SHOP_CA2,SHOP_CA3,CITY_EN,SHOP_CA1_EN,SHOP_CA2_EN,SHOP_CA3_EN
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点,huzhou,Delicious food,Leisure tea,Drinks \/ desserts
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN,harbin,Supermarket convenience store,Supermarket,NaN
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶,nanchang,Delicious food,Leisure tea,Tea with milk
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN,tianjin,Supermarket convenience store,Supermarket,NaN
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果,hangzhou,Delicious food,Snacks,Fresh fruit


In [162]:
#读入题目给出的类别信息表
category = pd.read_csv('./Flamingo_Rank4_code/data_new/SHOP_CAT.csv')
category.head()

,SHOP_CA1_EN,SHOP_CA2_EN,SHOP_CA3_EN,Num,CAT
0,Delicious food,Baking pastry,Bread,71,Baking pastry
1,Delicious food,Baking pastry,Cake,39,Baking pastry
2,Delicious food,Baking pastry,Other baked cakes,12,Baking pastry
3,Delicious food,Chinese food,Fujian Cuisine,1,Chinese food
4,Delicious food,Chinese food,Guangdong,12,Chinese food


In [163]:
 #对商家类别做独热编码,并整合进类别表
one_hot = pd.get_dummies(category['CAT'])
category = category.join(one_hot)
category.head()

,SHOP_CA1_EN,SHOP_CA2_EN,SHOP_CA3_EN,Num,CAT,Baking pastry,Chinese Fast Food,Chinese food,Convenience Store,Fresh fruit,Hot pot,Leisure tea,Non food,Noodle & Pastries,Other food,Other snacks,Porridge,Supermarket,barbecue,western-style fast food
0,Delicious food,Baking pastry,Bread,71,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Delicious food,Baking pastry,Cake,39,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Delicious food,Baking pastry,Other baked cakes,12,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Delicious food,Chinese food,Fujian Cuisine,1,Chinese food,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Delicious food,Chinese food,Guangdong,12,Chinese food,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [164]:
#给类别表中的列重命名
SHOP_SJ = map(lambda x:'SJ'+ str(x).zfill(2), np.arange(15))  #SJ01 SJ02 SJ03 .....SJ14
category.columns = ['SHOP_CA1_EN','SHOP_CA2_EN','SHOP_CA3_EN','Num', 'CAT'] + SHOP_SJ 

In [165]:
#将商家信息表与类别表整合
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN, category, on = ['SHOP_CA1_EN','SHOP_CA2_EN','SHOP_CA3_EN'],how = 'left')
SHOP_INFO_EN.head()

,SHOP_ID,CITY,SHOP_LOC,SHOP_PAY,SHOP_SCO,SHOP_COM,SHOP_LEV,SHOP_CA1,SHOP_CA2,SHOP_CA3,...,SJ05,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点,...,0,1,0,0,0,0,0,0,0,0
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,0,1,0,0
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶,...,0,1,0,0,0,0,0,0,0,0
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,0,1,0,0
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果,...,0,0,0,0,0,0,0,0,0,0


In [166]:
#根据地点聚合，计算每个地点有多少商家
SHOP_LOC_N = SHOP_INFO_EN.groupby('SHOP_LOC',as_index=False).count()
SHOP_LOC_N=SHOP_LOC_N.loc[:,['SHOP_LOC','SHOP_ID']].rename(columns = {'SHOP_ID':'SHOP_LOC_N'})
SHOP_LOC_N.head()

,SHOP_LOC,SHOP_LOC_N
0,1,1
1,2,1
2,3,2
3,4,2
4,5,1


In [167]:
#将每个地点的商家数整合进商家信息表
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,SHOP_LOC_N, on=['SHOP_LOC'], how= 'left')
SHOP_INFO_EN.head()

,SHOP_ID,CITY,SHOP_LOC,SHOP_PAY,SHOP_SCO,SHOP_COM,SHOP_LEV,SHOP_CA1,SHOP_CA2,SHOP_CA3,...,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14,SHOP_LOC_N
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点,...,1,0,0,0,0,0,0,0,0,2
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,1,0,0,1
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶,...,1,0,0,0,0,0,0,0,0,4
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,1,0,0,1
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果,...,0,0,0,0,0,0,0,0,0,1


In [168]:
#对商家信息表的空值进行填充，用平均数
SHOP_INFO_EN['SHOP_SCO'] = SHOP_INFO_EN['SHOP_SCO'].fillna(SHOP_INFO_EN['SHOP_SCO'].mean() )
SHOP_INFO_EN['SHOP_COM'] = SHOP_INFO_EN['SHOP_COM'].fillna(SHOP_INFO_EN['SHOP_COM'].mean() )

In [170]:
#提取并重命名商家信息表的列
SHOP_INFO_EN = SHOP_INFO_EN[['SHOP_ID','CITY_EN','SHOP_PAY','SHOP_SCO','SHOP_COM','SHOP_LEV','SHOP_LOC_N'] + SHOP_SJ]
SHOP_SD = map(lambda x:'SD'+ str(x).zfill(2), np.arange(5)) 
SHOP_INFO_EN.columns = ['SHOP_ID','CITY_EN'] + SHOP_SD + SHOP_SJ
SHOP_INFO_EN.head()

,SHOP_ID,CITY_EN,SD00,SD01,SD02,SD03,SD04,SJ00,SJ01,SJ02,...,SJ05,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14
0,1,huzhou,8,4.000000,12.000000,2,2,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,harbin,19,2.677004,3.130486,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,nanchang,5,3.000000,2.000000,0,4,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,tianjin,18,2.677004,3.130486,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,hangzhou,2,2.000000,2.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
#读入预先处理好的用户交易信息
PAYNW = pd.read_csv('./user_pay_new.csv')
PAYNW.head() #

,SHOP_ID,DATE,HOUR,Num_raw,Num_post,DofW
0,1,2015-10-10,15,14,14.0,5
1,1,2015-10-10,16,20,20.0,5
2,1,2015-10-10,17,16,16.0,5
3,1,2015-10-10,18,24,24.0,5
4,1,2015-10-10,19,48,48.0,5


In [172]:
#读入预先处理好的用户浏览信息
VIENW = pd.read_csv('./user_view_new.csv')
VIENW.head()

,SHOP_ID,DATE,HOUR,Num_raw,Num_post,DofW
0,1,2016-02-01,10,1,1.0,0
1,1,2016-02-01,14,2,2.0,0
2,1,2016-02-01,17,1,1.0,0
3,1,2016-02-01,19,1,1.0,0
4,1,2016-02-01,20,1,1.0,0


In [173]:
#读入节日信息（周末、假日为1）
HOLI = pd.read_csv('./Flamingo_Rank4_code/additional/HOLI.csv')
HOLI['DATE'] = [ (lambda x: str(datetime.datetime.strptime(str(x),'%Y%m%d').date() ) )(x) for x in HOLI['DATE'] ] 
HOLI.head()

,DATE,HOLI
0,2015-06-26,0
1,2015-06-27,1
2,2015-06-28,1
3,2015-06-29,0
4,2015-06-30,0


In [174]:
#计算商家每天销量最好的一小时
TOP_N = 1
SHOP_ID = []
SHOP_HOUR_head = []
SHOP_PCT_head = []
for SHOP_IND in range(1,2001):
    tt = PAYNW[PAYNW['SHOP_ID'] == SHOP_IND] #提取一个商家的用户客流信息
    tt2 = tt.groupby('HOUR',as_index = False).sum() #按小时分组，并求和
    tt3 = tt2.sort_values('Num_post',ascending = False,inplace = False) #按客流数降序排序
    tt4 = tt3.head(TOP_N)['HOUR'].values #获得客流量最大的是哪一小时
    tt5 = tt3.head(TOP_N)['Num_post'].values/tt3['Num_post'].sum() #计算这一小时的客流量在一天中总客流量的比例
    SHOP_ID.append(SHOP_IND)
    SHOP_HOUR_head.append(tt4)
    SHOP_PCT_head.append(tt5)#所占百分比

#数组转df并连接
SHOP_ID_df = pd.DataFrame(SHOP_ID)
SHOP_HOUR_head_df = pd.DataFrame(SHOP_HOUR_head)
SHOP_PCT_head_df = pd.DataFrame(SHOP_PCT_head)
SELL_INFO = pd.concat([SHOP_ID_df,SHOP_HOUR_head_df,SHOP_PCT_head_df],axis = 1)
#改列名
SHOP_SE =  [(lambda x:('SE'+ str(x).zfill(2))) (x)  for x in range(TOP_N)]
SHOP_SF =  [(lambda x:('SF'+ str(x).zfill(2))) (x)  for x in range(TOP_N)]
SELL_INFO.columns = ['SHOP_ID'] + SHOP_SE + SHOP_SF
 
SELL_INFO.head() #se表示最好的那一小时 sf表示该小时客流量占当天的百分比

,SHOP_ID,SE00,SF00
0,1,19,0.104353
1,2,19,0.114248
2,3,19,0.123096
3,4,19,0.163664
4,5,12,0.132072


In [175]:
# 计算商铺其他信息
SHOP_ID = []
SHOP_OPEN = []
SHOP_CLOSE = []
SHOP_LAST = []
SHOP_MEAN = []
for SHOP_IND in range(1,2001):
    tt = PAYNW[PAYNW['SHOP_ID'] == SHOP_IND]
    tt2 = tt.groupby('DATE',as_index = False).min().mean()#找出该商铺平均在几点开业
    tt3 = tt.groupby('DATE',as_index = False).max().mean()#找出该商铺平均在几点歇业
    tt['MEAN']  = tt['Num_post'] * tt['HOUR'] 
    SHOP_ID.append(SHOP_IND)    
    SHOP_OPEN.append(tt2.HOUR)
    SHOP_CLOSE.append(tt3.HOUR)
    SHOP_LAST.append(tt3.HOUR -tt2.HOUR   )  #营业时长
    SHOP_MEAN.append(tt['MEAN'].sum()/tt['Num_post'].sum() ) #平均值？

SHOP_ID_df = pd.DataFrame(SHOP_ID)
SHOP_OPEN_df = pd.DataFrame(SHOP_OPEN )
SHOP_CLOSE_df = pd.DataFrame(SHOP_CLOSE )   
SHOP_LAST_df = pd.DataFrame(SHOP_LAST )
SHOP_MEAN_df = pd.DataFrame(SHOP_MEAN )
HOUR_INFO = pd.concat([SHOP_ID_df,SHOP_OPEN_df,SHOP_CLOSE_df, SHOP_LAST_df,SHOP_MEAN_df],axis = 1)

SHOP_SG = map(lambda x:'SG'+ str(x).zfill(2), np.arange(4))
HOUR_INFO.columns = ['SHOP_ID'] + SHOP_SG 

HOUR_INFO.head() #营业时间、歇业时间、营业时长、平均销量（？）

/home/zlq/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,SHOP_ID,SG00,SG01,SG02,SG03
0,1,6.961240,22.046512,15.085271,15.164778
1,2,8.343465,20.942249,12.598784,15.509322
2,3,10.198529,21.676471,11.477941,16.381929
3,4,8.466667,20.942857,12.476190,16.315445
4,5,9.599010,20.727723,11.128713,15.832341


In [176]:

PAYNW_gp = PAYNW.groupby(['DATE','SHOP_ID'],as_index = False).sum()#计算一天中某商铺总共的客流量
PAYNW_gp['DofW'] = Datestr2DofW(PAYNW_gp['DATE'] ) #将日期转为周几
PAYNW_gp = pd.merge(PAYNW_gp,HOLI,on = ['DATE'],how = 'left') #根据日期添加节日信息
PAYNW_gp.head()#重要的是num_post

,DATE,SHOP_ID,HOUR,Num_raw,Num_post,DofW,HOLI
0,2015-06-26,145,17,1,1.0,4,0
1,2015-06-26,260,6,2,2.0,4,0
2,2015-06-26,560,9,1,1.0,4,0
3,2015-06-26,897,151,27,27.0,4,0
4,2015-06-26,1332,208,35,35.0,4,0


In [177]:
#生成商铺某天的浏览总量表，并添加周几、节日信息
VIENW_gp = VIENW.groupby(['DATE','SHOP_ID'],as_index = False).sum()
VIENW_gp['DofW'] = Datestr2DofW(VIENW_gp['DATE'] )
VIENW_gp = pd.merge(VIENW_gp,HOLI,on = ['DATE'],how = 'left')
VIENW_gp.head() #重要的是num_post

,DATE,SHOP_ID,HOUR,Num_raw,Num_post,DofW,HOLI
0,2016-02-01,1,80,6,6.000000,0,0
1,2016-02-01,2,264,244,243.584963,0,0
2,2016-02-01,6,69,4,4.000000,0,0
3,2016-02-01,7,243,71,68.339850,0,0
4,2016-02-01,9,139,15,15.000000,0,0


In [178]:
#计算浏览量与成交量之间的比率
PAYNW_VIENW = pd.merge(PAYNW_gp,VIENW_gp, on =['DATE','SHOP_ID'], how = 'inner' )
PAYNW_VIENW['RATIO'] = PAYNW_VIENW['Num_post_y'] / PAYNW_VIENW['Num_post_x']
SHOP_PAYNW_VIENW = PAYNW_VIENW.groupby('SHOP_ID',as_index = False).mean()
SHOP_SC = 'SC00'    
RATIO_INFO = SHOP_PAYNW_VIENW[['SHOP_ID','RATIO']].rename(columns = {'RATIO':SHOP_SC})
RATIO_INFO.head()### sc表示浏览交易比

,SHOP_ID,SC00
0,1,0.121528
1,2,0.429469
2,3,0.063027
3,4,0.358786
4,5,0.334671


In [179]:
#计算商家的上线日期
GAP_INFO = PAYNW.groupby('SHOP_ID',as_index = False).min().loc[:,['SHOP_ID','DATE']]
GAP_INFO['RAT_DAY'] = pd.to_datetime(  GAP_INFO['DATE']) - datetime.date(2015,6,26)#上线日期距离2015.6.26有多少天
GAP_INFO['RAT_DAY'] = [(lambda x:(x.days)) (x) for x in GAP_INFO['RAT_DAY'] ]

#重命名列
SHOP_SH =  [(lambda x:('SH'+ str(x).zfill(2))) (x)  for x in range(2)]
GAP_INFO.columns = ['SHOP_ID'] + SHOP_SH

GAP_INFO.head()

,SHOP_ID,SH00,SH01
0,1,2015-10-10,106
1,2,2015-11-25,152
2,3,2016-06-18,358
3,4,2016-07-19,389
4,5,2015-09-28,94


In [184]:
#商家正常工作日、节假日客流量的中位数

PAYNW_gp_wd = PAYNW_gp[PAYNW_gp['HOLI']==0].groupby('SHOP_ID',as_index = False).median()
PAYNW_gp_wd = PAYNW_gp_wd[['SHOP_ID','Num_post']].rename(columns = {'Num_post':'WD_MED'})

PAYNW_gp_wk = PAYNW_gp[PAYNW_gp['HOLI']>0].groupby('SHOP_ID',as_index = False).median()
PAYNW_gp_wk = PAYNW_gp_wk[['SHOP_ID','Num_post']].rename(columns = {'Num_post':'WK_MED'})

PAYNW_gp_wdwk = pd.merge(PAYNW_gp_wd, PAYNW_gp_wk, on='SHOP_ID',how = 'left')
PAYNW_gp_wdwk.head()

,SHOP_ID,WD_MED,WK_MED
0,1,227.0,231.0
1,2,82.0,117.0
2,3,68.0,92.5
3,4,105.0,194.0
4,5,198.5,114.0


In [185]:
#按照周几和假日计算一星期中每天客流量的中位数
HOLI_list = [0,0,0,0,0,1,1]
DofW_list = [0,1,2,3,4,5,6]

for holi_ind, DofW_ind in zip(HOLI_list,DofW_list):
    DAYOFWEEk = PAYNW_gp[(PAYNW_gp['HOLI']==holi_ind)&(PAYNW_gp['DofW']==DofW_ind)].groupby('SHOP_ID',as_index = False).median()
    DAYOFWEEk = DAYOFWEEk[['SHOP_ID','Num_post']].rename(columns = {'Num_post':'d'+str(DofW_ind)})
    PAYNW_gp_wdwk = pd.merge(PAYNW_gp_wdwk, DAYOFWEEk, on='SHOP_ID',how = 'left')
PAYNW_gp_wdwk.head()

,SHOP_ID,WD_MED,WK_MED,d0,d1,d2,d3,d4,d5,d6
0,1,227.0,231.0,218.0,222.000000,219.0,224.0,248.292481,250.000000,228.000000
1,2,82.0,117.0,81.0,79.292481,83.0,82.0,91.000000,111.000000,118.000000
2,3,68.0,92.5,63.0,61.500000,63.5,68.0,81.000000,97.000000,85.000000
3,4,105.0,194.0,107.5,90.000000,116.0,104.0,110.000000,199.292481,197.792481
4,5,198.5,114.0,224.5,184.500000,210.0,219.0,119.500000,164.000000,132.500000


In [186]:
#通过除平均值，对每日客流量进行归一化
SHOP_SI =  [(lambda x:('SI'+ str(x).zfill(2))) (x)  for x in range(9)]
PAYNW_gp_wdwk.columns = ['SHOP_ID'] + SHOP_SI

PAYNW_gp_wdwk['FIX'] = PAYNW_gp_wdwk[SHOP_SI].mean(axis = 1)
PAYNW_gp_wdwk[SHOP_SI] = PAYNW_gp_wdwk[SHOP_SI].div( PAYNW_gp_wdwk['FIX'],axis = 0)
del PAYNW_gp_wdwk['FIX']

PAYNW_gp_wdwk['ratio'] = PAYNW_gp_wdwk['SI00']/ PAYNW_gp_wdwk['SI01']#表示工作日与假日客流量的比
SHOP_SI =  [(lambda x:('SI'+ str(x).zfill(2))) (x)  for x in range(10)]
PAYNW_gp_wdwk.columns = ['SHOP_ID'] + SHOP_SI
PAYNW_gp_wdwk.head()

,SHOP_ID,SI00,SI01,SI02,SI03,SI04,SI05,SI06,SI07,SI08,SI09
0,1,0.988249,1.005663,0.949067,0.966482,0.953421,0.975189,1.080946,1.088380,0.992603,0.982684
1,2,0.874105,1.247198,0.863445,0.845243,0.884764,0.874105,0.970043,1.183239,1.257858,0.700855
2,3,0.900662,1.225166,0.834437,0.814570,0.841060,0.900662,1.072848,1.284768,1.125828,0.735135
3,4,0.772321,1.426954,0.790709,0.661989,0.853230,0.764965,0.809098,1.465883,1.454850,0.541237
4,5,1.140440,0.654963,1.289818,1.060006,1.206511,1.258219,0.686562,0.942228,0.761251,1.741228


In [187]:
#将以上特征进行整合
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,SELL_INFO, on='SHOP_ID',how = 'left')   #SA
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,HOUR_INFO, on='SHOP_ID',how = 'left')  # SE,SF
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,RATIO_INFO, on='SHOP_ID',how = 'left')  #SC
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,GAP_INFO, on='SHOP_ID',how = 'left')    #SH
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN,PAYNW_gp_wdwk, on='SHOP_ID',how = 'left')    #SI
SHOP_INFO_EN = SHOP_INFO_EN.fillna(0)
SHOP_INFO_EN.head()

,SHOP_ID,CITY_EN,SD00,SD01,SD02,SD03,SD04,SJ00,SJ01,SJ02,...,SI00,SI01,SI02,SI03,SI04,SI05,SI06,SI07,SI08,SI09
0,1,huzhou,8,4.000000,12.000000,2,2,0,0,0,...,0.988249,1.005663,0.949067,0.966482,0.953421,0.975189,1.080946,1.088380,0.992603,0.982684
1,2,harbin,19,2.677004,3.130486,1,1,0,0,0,...,0.874105,1.247198,0.863445,0.845243,0.884764,0.874105,0.970043,1.183239,1.257858,0.700855
2,3,nanchang,5,3.000000,2.000000,0,4,0,0,0,...,0.900662,1.225166,0.834437,0.814570,0.841060,0.900662,1.072848,1.284768,1.125828,0.735135
3,4,tianjin,18,2.677004,3.130486,1,1,0,0,0,...,0.772321,1.426954,0.790709,0.661989,0.853230,0.764965,0.809098,1.465883,1.454850,0.541237
4,5,hangzhou,2,2.000000,2.000000,0,1,0,0,0,...,1.140440,0.654963,1.289818,1.060006,1.206511,1.258219,0.686562,0.942228,0.761251,1.741228


In [188]:
SHOP_INFO_EN.to_csv('SHOP_FEATURES.csv',index = False)